# Computing the bicrystallographic aspects of Misorientations (FCC lattice)

In [1]:
# import sys
import gbpy.byxtal.lattice as gbl;
import gbpy.byxtal.csl_utility_functions as cuf;
import gbpy.byxtal.find_csl_dsc as fcd;
import numpy as np;
import numpy.linalg as nla
import gbpy.byxtal.tools as bxt;
import gbpy.byxtal.misorient_fz as mfz;
import gbpy.byxtal.disorient_symm_props as dsp;
# from sympy.matrices import Matrix, eye, zeros;

Define the lattice: Here we use the `lattice` module and use the default option. The default option will initialize a cubic lattice with $a = 1$.

In [14]:
l1 = gbl.Lattice('hP_Id')
sig_type = 'common'
l_p_po = l1.l_p_po
print(l_p_po)

[[ 1.        -0.5        0.       ]
 [ 0.         0.8660254  0.       ]
 [ 0.         0.         1.633    ]]


First, initialize the $\Sigma$ number of interest. Note that for cubic crystals, CSL rotations exist only for odd-values of $\Sigma$.

In [3]:
sig_num = 7

The function `csl_rotations` in the module `gbpy.byxtal.csl_utility_functions`, will compute the $\Sigma$ rotation (a misorientation) in the reference frame of the primitive cell.

In [5]:
s1 = cuf.csl_rotations(sig_num, sig_type, l1);
print(s1)

{'N': array([[[ 8., -3.,  0.],
        [ 3.,  5.,  0.],
        [ 0.,  0.,  7.]]]), 'D': array([[[7., 7., 7.],
        [7., 7., 7.],
        [7., 7., 7.]]])}


As can be obsrved from the output above, there exist multiple rotations that correspond to a given $\Sigma$ number. This is usually the case when $\Sigma$ is large. In the following, we will compute the bycrystallography properties for one of the misorientations (e.g. corresponding to the second rotation in the array). The bycrystallography properties of interest are as follows:
+ The Coincidence Site Lattice (CSL). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_2.html).
+ The DSC lattice that defines the **D**isplacements that are **S**ymmetry **C**onserving (also referred to as the **D**isplacement **S**hift **C**omplete). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_3.html)
+ The boundary-plane symmetry group, and the symmetry axes. For more details refer to the following article:
    - [**Symmetries in the representation of grain boundary-plane distributions**](https://www.tandfonline.com/doi/full/10.1080/14786435.2012.722700?casa_token=4nDY3LLK9vQAAAAA%3AgECePf6JCdOPx6K_7cfxrJATdUAwMOSe3-F5NIdpjuI26uAPQfkkSTa61np1AWoDGQ3wcJrh3SS9). Patala, S., & Schuh, C. A. (2013). . Philosophical Magazine, 93(5), 524-573.

In [6]:
ct1 = 1
sig_id = str(sig_num)+'_'+str(ct1)
print(sig_id)

7_1


The $\Sigma$-misorientation is a rotation operation that is defined in the $P$ lattice reference frame and is computed as shown in the code snippet below. We use the variable `T_p1top2_p1` denote the **T**ransformation (`T`) from the $P1$ lattice to the $P2$ lattice (`p1top2`) expressed in the $P1$ reference frame (`p1`).

In [7]:
T_p1top2_p1 = s1['N'][ct1-1]/s1['D'][ct1-1]
print(T_p1top2_p1)

[[ 1.14285714 -0.42857143  0.        ]
 [ 0.42857143  0.71428571  0.        ]
 [ 0.          0.          1.        ]]


The **CSL** and the **DSC** lattice are determined using the `find_csl_dsc` function in the `byxtal.find_csl_dsc` module. The lattices are expressed using the basis vectors of the primitive cell $(P1)$. The following varibales are used:
+ `l_csl_p`: The lattice (`l`) of the CSL (`csl`) expressed in the primitive reference frame (`p`).
+ `l_dsc_p`: The lattice (`l`) of the DSC (`dsc`) expressed in the primitive reference frame (`p`).

In [9]:
T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
TI_p1top2_p1 = T_p1top2_p1*sig_num
TI_p1top2_p1 = (np.around(TI_p1top2_p1)).astype(int)
# l_csl_p = fcd.csl_finder(TI_p1top2_p1, sig_num, l_p_po, 1e-6)
l_csl_p = fcd.csl_finder_wSigma(TI_p1top2_p1, sig_num, l_p_po, 1e-6)
print(l_csl_p)


[[-2.  0.  1.]
 [ 1.  0.  3.]
 [ 0.  1.  0.]]


In [10]:
l_csl_p1 = l_csl_p.copy()
l_p2_p1 = T_p1top2_p1.copy()
l_p1_p2 = nla.inv(l_p2_p1)
l_csl_p2 = l_p1_p2.dot(l_csl_p1)
print(l_csl_p2)

[[-1.  0.  2.]
 [ 2.  0.  3.]
 [ 0.  1.  0.]]


In [12]:
fcd.check_csl(l_csl_p, l_p_po, T_p1top2_p1, sig_num, True)

l_csl_po is defined in the l_p1_po lattice
l_csl_po is defined in the l_p2_po lattice
V(csl_po)/V(p1_po) = Sigma =  7


True